# Food in Art

### Angles

- Proportion of food related items in museums
- Proportion of food related artworks by epoch and country
- Is food central topic
- Network diagram linking culinary styles with corresponding art movements.
- fruit, meal, dessert, etc
- feasts, banquets, etc
- food as a medium?
- food symbolism?
- **Analysis Angle:** Investigate how the introduction of new food items through trade influenced their depiction in art.
    - **Visualization:** Maps showing trade routes alongside corresponding changes in food representation in art."
- compare current food focus by country
- Nutrional evolution
- **Analysis Angle:** Explore how periods of famine or abundance influenced the portrayal of food in art.
    - **Visualization:** Correlation charts linking historical events of scarcity or abundance with changes in food depiction."
- Social classes?
- Agriculture, markets?
- **Analysis Angle:** Analyze the use of color in food-themed art and its psychological effects on viewers.
- **Visualization:** Color wheel charts displaying dominant hues in various artworks."
- **Analysis Angle:** Investigate how chefs create dishes inspired by artworks, merging visual art with gastronomy.
- **Visualization:** Side-by-side images of artworks and the culinary dishes they inspired."

In [1]:
import pywikibot
from pywikibot import pagegenerators
import xml.etree.ElementTree as ET
import pandas as pd
from lxml import etree
import re
from urllib.parse import quote

## TODO
1. Set columns : AUTHOR TITLE DATE ORIGIN_COUNTRY DISPLAY_COUNTRY DISPLAY_LOCATION TYPE SCHOOL TIMEFRAME? TIME_PERIOD  WA_URL WIKI_URL WIKIDATA_ID IMAGE_URL KEYWORDS
2. Get data from web gallery
3. Clean and filter data, only paintings, remove dupes, remove (details) cf mona lisa
4. Scrape web gallery for description and image url
5. Save to CSV, cache
6. fetch all images
5. Get data from wikidata
7. supplement first df
6. remove dupes
4. fine tune yolo5 for food recognition using wikidata

## Data exploration

In [ ]:
paintings_df = pd.read_csv('data/paintings_with_descriptions.csv')
paintings_df

In [ ]:
#get infos
display(paintings_df.info())
display(paintings_df.isna().sum())


In [4]:
# drop duplicates
paintings_df = paintings_df.drop_duplicates(subset=['item'])
paintings_df = paintings_df.drop_duplicates(subset=['image_url'])

In [ ]:
# remove missing images
paintings_df = paintings_df.dropna(subset=['image_url'])


print(paintings_df.shape)
display(paintings_df.info())
display(paintings_df)


## NLP

In [6]:
classes = ['wine', 'beverage', 'meat', 'fruit', 'vegetable', 'bread', 'dairy', 'dessert', 'seafood']